In [1]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from langchain_openai import OpenAI  
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

In [2]:
load_dotenv()

True

In [16]:
# Ferramentas
def get_weather(city: str) -> str:
    return f"O clima em {city} está ensolarado com 25°C."

class WeatherInput(BaseModel):
    city: str

weather_tool = StructuredTool.from_function(
    func=get_weather, name="WeatherInfo",
    description="Obter clima de uma cidade",
    args_schema=WeatherInput,
)

def convert_currency(from_currency: str, to_currency: str) -> str:
    return f"1 {from_currency} = 5.42 {to_currency}"

class CurrencyInput(BaseModel):
    from_currency: str
    to_currency: str

currency_tool = StructuredTool.from_function(
    func=convert_currency, name="CurrencyConverter",
    description="Converte moeda de um código para outro",
    args_schema=CurrencyInput,
)

# LLM e agente
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=[weather_tool, currency_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True  # importante
)

# Rodando o agente
query = "Qual o clima em São Paulo e quanto vale 1 USD em BRL?"
result = agent({ "input": query })

# Saída
print("🎯 Resposta final:", result["output"])
print("\n🧠 Pensamentos e ações:")
for action, obs in result["intermediate_steps"]:
    print(f"- Pensou: usar {action.tool} com input {action.tool_input}")
    print(f"  → Resultado: {obs}")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "WeatherInfo",
  "action_input": {
    "city": "São Paulo"
  }
}
```


Observation: O clima em São Paulo está ensolarado com 25°C.
Thought: Agora, vamos converter a moeda.
Action:
```
{
  "action": "CurrencyConverter",
  "action_input": {
    "from_currency": "USD",
    "to_currency": "BRL"
  }
}
```


Observation: 1 USD = 5.42 BRL
Thought: Agora, vamos responder à pergunta do usuário.
Action:
```
{
  "action": "Final Answer",
  "action_input": "O clima em São Paulo está ensolarado com 25°C e 1 USD vale 5.42 BRL."
}
```



> Finished chain.
🎯 Resposta final: O clima em São Paulo está ensolarado com 25°C e 1 USD vale 5.42 BRL.

🧠 Pensamentos e ações:
- Pensou: usar WeatherInfo com input {'city': 'São Paulo'}
  → Resultado: O clima em São Paulo está ensolarado com 25°C.
- Pensou: usar CurrencyConverter com input {'from_currency': 'USD', 'to_currency': 'BRL'}
  → Resultado: 1 USD = 5.42 BRL


In [26]:
for action, obs in result["intermediate_steps"]:
    print(f"- Pensou: usar ferramenta '{action.tool}' com input '{action.tool_input}'")
    print(f"  → Observação retornada: {obs}")

- Pensou: usar ferramenta 'WeatherInfo' com input '{'city': 'São Paulo'}'
  → Observação retornada: O clima em São Paulo está ensolarado com 25°C.
- Pensou: usar ferramenta 'CurrencyConverter' com input '{'from_currency': 'USD', 'to_currency': 'BRL'}'
  → Observação retornada: 1 USD = 5.42 BRL


In [23]:
for action, obs in result["intermediate_steps"]:
    print("🧠 Thought + Action log:")
    print(action.log)           # Aqui está seu Thought + Action + Action Input
    print("🎯 Observation:", obs)
    print("---")


🧠 Thought + Action log:
Action:
```
{
  "action": "WeatherInfo",
  "action_input": {
    "city": "São Paulo"
  }
}
```


🎯 Observation: O clima em São Paulo está ensolarado com 25°C.
---
🧠 Thought + Action log:
 Agora, vamos converter a moeda.
Action:
```
{
  "action": "CurrencyConverter",
  "action_input": {
    "from_currency": "USD",
    "to_currency": "BRL"
  }
}
```


🎯 Observation: 1 USD = 5.42 BRL
---


In [25]:
for action, obs in result["intermediate_steps"]:
    log = action.log
    thought = log.split("\nAction:")[0]  # pega tudo antes de "Action:"
    print("💭 Thought:", thought.strip())
    print("🎯 Observation:", obs)
    print("---")


💭 Thought: Action:
```
{
  "action": "WeatherInfo",
  "action_input": {
    "city": "São Paulo"
  }
}
```
🎯 Observation: O clima em São Paulo está ensolarado com 25°C.
---
💭 Thought: Agora, vamos converter a moeda.
🎯 Observation: 1 USD = 5.42 BRL
---
